<a href="https://colab.research.google.com/github/anscc2/predict_app/blob/main/DeteksiBully_CVPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [2]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("GPU is available")
else:
  device = torch.device("cpu")
  print("GPU not available, CPU used")

GPU is available


# Load Dataset

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Dataset FIX/dataset_clean2.csv')
data

,Tweet,Label,preprocessing_text,text_len
0,@intermilanstory Pelatih bangsat memang ente I...,1,latih bangsat memang kamu inzanah rasa jeruk k...,24
1,bangsat temen gue dimana,0,bangsat teman,2
2,@keuangannews_id Rasmus paludan harus nya di b...,1,rasmus paludan nya bakar hidup mati gosong sih...,9
3,@AnthonyBudiawan Memang babi ini negara.. peja...,1,memang babi negara jabat bangsat,5
4,@bajibekasii Mampus lu bangsat,1,mampus kamu bangsat,3
...,...,...,...,...
2393,@sosmedkeras Cowok bejad.bajingan. kualat lu n...,1,cowok bejat bajing kualat kamu kasihan sangat ...,9
2394,@WagimanDeep212_ Tampang sich rupa manusia..ta...,1,tampang sih rupa manusia laku bejat,6
2395,"@zarnaout45 Pejabat gk punya Adab, mikirin nya...",1,jabat punya adab pikir nya cuma buang oli doan...,15
2396,@afilrifak Ini yg bejad pejabatnya yg bantuin ...,1,bejat jabat bantu sih bos kalau jabat kadrun s...,21


In [4]:
texts = data['preprocessing_text'].values
labels = data['Label']

In [5]:
labels.value_counts()

0    1326
1    1072
Name: Label, dtype: int64

# Tokenize

In [6]:
EMBEDDING_DIM = 300
MAXLEN = 52
PADDING = 'post'
OOV_TOKEN = "<OOV>"

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def fit_tokenizer(train_sentences, oov_token):
    tokenizer = Tokenizer(oov_token=oov_token)
    tokenizer.fit_on_texts(train_sentences)
    return tokenizer

def seq_pad(sentences, tokenizer, padding, maxlen):
    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    # Pad the sequences using the correct padding and maxlen
    pad_sequence = pad_sequences(sequences, maxlen=maxlen, padding=padding)
    return pad_sequence

tokenizer = fit_tokenizer(texts, OOV_TOKEN)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
print(f"Vocabulary contains {vocab_size} words\n")

texts_pad_seq = seq_pad(texts, tokenizer, PADDING, MAXLEN)
print(f"Padded testing sequences have shape: {texts_pad_seq.shape}")

Vocabulary contains 3642 words

Padded testing sequences have shape: (2398, 52)


In [44]:
texts[77]

'bangsat orang jelas sangat kemarin tuduh bicara jelek padahal nyata terus sekarang bicara kalau henti teman anjing aneh kamu'

In [8]:
texts_pad_seq[77]

array([ 16,   3,  91,   4, 240, 643,  84,  23, 103, 117,  63,  86,  84,
         5, 726,  78,   7, 330,   2,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [9]:
# Menyimpan word_index kedalam sebuah file json
import json
with open('/content/word_index.json', 'w') as fp:
    json.dump(word_index, fp)

# Load Pretrained Vectors

In [10]:
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 6.1 MB/s eta 0:00:00


In [11]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199773 sha256=d95cae5366a93151691af2d160689eec41458ba59abf9b538b55f5c6f7eed6ad
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [12]:
from huggingface_hub import hf_hub_download
import fasttext

model_fasttext = fasttext.load_model(hf_hub_download("facebook/fasttext-id-vectors", "model.bin"))

model.bin:   0%|          | 0.00/7.24G [00:00<?, ?B/s]

In [13]:
def create_embedding_matrix(word_index, embedding_dict, d_model=300):
    embedding_matrix = np.zeros((len(word_index)+1, d_model))
    for word, i in word_index.items():
      embedding_vector = embedding_dict[word]
      if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [14]:
# Load pretrained vectors
embeddings = create_embedding_matrix(word_index, model_fasttext)
embeddings = torch.tensor(embeddings)

In [15]:
embeddings

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0012,  0.0030, -0.0020,  ...,  0.0017,  0.0037,  0.0004],
        [ 0.0671, -0.0287,  0.0505,  ..., -0.1375,  0.0357,  0.1539],
        ...,
        [ 0.0301, -0.0198,  0.0725,  ..., -0.0711,  0.0070,  0.0692],
        [ 0.0444, -0.0392,  0.1819,  ...,  0.0308, -0.0019,  0.0810],
        [-0.0125, -0.0359, -0.0318,  ..., -0.1402,  0.0024,  0.0230]],
       dtype=torch.float64)

In [38]:
# torch.save(embeddings, 'embeddings_matrix.pth')

# LSTM Model

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, pretrained_embedding=None, hidden_dim=None, dropout=None, bidirectional=None):
      super(LSTMModel, self).__init__()
      # Embedding Layer
      self.vocab_size, self.embed_dim = pretrained_embedding.shape
      self.embedding = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)

      self.dropout = nn.Dropout(dropout)

      self.lstm = nn.LSTM(input_size=self.embed_dim,
                          hidden_size=hidden_dim,
                          batch_first=True,
                          bidirectional=bidirectional)

      self.avg_pooling = nn.AdaptiveAvgPool1d(1)
      self.max_pooling = nn.AdaptiveMaxPool1d(1)

      # Concatenate Layer
      if bidirectional:
        self.fc = nn.Linear(hidden_dim*2*2, 1)
      else:
        self.fc = nn.Linear(hidden_dim*2, 1)

      self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      x = self.embedding(x).float()
      x = self.dropout(x)
      x, _ = self.lstm(x)
      x = self.dropout(x)
      avg_pooled = self.avg_pooling(x.permute(0, 2, 1)).view(x.size(0), -1)
      max_pooled = self.max_pooling(x.permute(0, 2, 1)).view(x.size(0), -1)
      concat = torch.cat((avg_pooled, max_pooled), dim=1)
      out = self.fc(concat)
      return self.sigmoid(out)

def initilize_lstm_model(pretrained_embedding=None,
                        hidden_dim=128,
                        dropout=0.3,
                        bidirectional=True,
                        learning_rate=0.001):

    lstm_model = LSTMModel(pretrained_embedding=pretrained_embedding,
                           hidden_dim=hidden_dim,
                           dropout=dropout,
                           bidirectional=bidirectional)
    lstm_model.to(device)
    lstm_optimizer = optim.RMSprop(lstm_model.parameters(), lr=learning_rate, weight_decay=0.01)
    return lstm_model, lstm_optimizer

# GRU Model

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class GRUModel(nn.Module):
    def __init__(self, pretrained_embedding=None, hidden_dim=None, dropout=None, bidirectional=None):
      super(GRUModel, self).__init__()
      # Embedding Layer
      self.vocab_size, self.embed_dim = pretrained_embedding.shape
      self.embedding = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)

      self.dropout = nn.Dropout(dropout)

      self.gru = nn.GRU(input_size=self.embed_dim,
                          hidden_size=hidden_dim,
                          batch_first=True,
                          bidirectional=bidirectional)

      self.avg_pooling = nn.AdaptiveAvgPool1d(1)
      self.max_pooling = nn.AdaptiveMaxPool1d(1)

      # Concatenate Layer
      if bidirectional:
        self.fc = nn.Linear(hidden_dim*2*2, 1)
      else:
        self.fc = nn.Linear(hidden_dim*2, 1)

      self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      x = self.embedding(x).float()
      x = self.dropout(x)
      x, _ = self.gru(x)
      x = self.dropout(x)
      avg_pooled = self.avg_pooling(x.permute(0, 2, 1)).view(x.size(0), -1)
      max_pooled = self.max_pooling(x.permute(0, 2, 1)).view(x.size(0), -1)
      concat = torch.cat((avg_pooled, max_pooled), dim=1)
      out = self.fc(concat)
      return self.sigmoid(out)

def initilize_gru_model(pretrained_embedding=None,
                        hidden_dim=128,
                        dropout=0.3,
                        bidirectional=True,
                        learning_rate=0.001):

    gru_model = GRUModel(pretrained_embedding=pretrained_embedding,
                           hidden_dim=hidden_dim,
                           dropout=dropout,
                           bidirectional=bidirectional)
    gru_model.to(device)
    gru_optimizer = optim.RMSprop(gru_model.parameters(), lr=learning_rate, weight_decay=0.01)
    return gru_model, gru_optimizer

# SRU Model

In [19]:
!pip install sru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 7.9 MB/s eta 0:00:00


In [25]:
from sru import SRU

class SRUModel(nn.Module):
    def __init__(self, pretrained_embedding=None, hidden_dim=None, dropout=None, bidirectional=None):
        super(SRUModel, self).__init__()
        # Embedding Layer
        self.vocab_size, self.embed_dim = pretrained_embedding.shape
        self.embedding = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)

        self.dropout = nn.Dropout(dropout)
        # SRU Layer
        self.sru = SRU(input_size=self.embed_dim,
                       hidden_size=hidden_dim,
                       num_layers=1,
                       bidirectional=bidirectional)

        self.avg_pooling = nn.AdaptiveAvgPool1d(1)
        self.max_pooling = nn.AdaptiveMaxPool1d(1)

        # Concatenate Layer
        if bidirectional:
          self.fc = nn.Linear(hidden_dim*2*2, 1)
        else:
          self.fc = nn.Linear(hidden_dim*2, 1)

        # Output Layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      x = self.embedding(x).float()
      x = self.dropout(x)
      x, _ = self.sru(x)
      x = self.dropout(x)
      avg_pooled = self.avg_pooling(x.permute(0, 2, 1)).view(x.size(0), -1)
      max_pooled = self.max_pooling(x.permute(0, 2, 1)).view(x.size(0), -1)
      concat = torch.cat((avg_pooled, max_pooled), dim=1)
      out = self.fc(concat)
      return self.sigmoid(out)

def initilize_sru_model(pretrained_embedding=None,
                        hidden_dim=128,
                        dropout=0.3,
                        learning_rate=0.001,
                        bidirectional=True):

    sru_model = SRUModel(pretrained_embedding=pretrained_embedding,
                         hidden_dim=hidden_dim,
                         dropout=dropout,
                         bidirectional=bidirectional)
    sru_model.to(device)
    sru_optimizer = optim.RMSprop(sru_model.parameters(), lr=learning_rate, weight_decay=0.01)
    return sru_model, sru_optimizer

# Train CV

In [21]:
import random

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def reset_weights(m):
  ''' Try resetting model weights to avoid weight leakage. '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
      layer.reset_parameters()

In [22]:
from sklearn.model_selection import StratifiedKFold

k_folds = 5
loss_function = nn.BCELoss()

# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

tensor_inputs = torch.tensor(texts_pad_seq)
tensor_labels = torch.tensor(np.array(labels))
dataset_tensor = torch.utils.data.TensorDataset(tensor_inputs, tensor_labels)

In [42]:
def train_cv(network, optimizer, dataset, fold, batch_size, num_epochs):
  results = {}

  print("-"*60)
  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset.tensors[0], dataset.tensors[1])):
    print(f'FOLD {fold+1}')
    print("-"*60)

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_subsampler)

    network.apply(reset_weights)
    for epoch in range(0, num_epochs):
      total_loss, correct, total = 0, 0, 0
      network.train()
      for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = loss_function(outputs.view(-1), targets.float())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        predicted = (outputs > 0.5).float()
        total += targets.size(0)
        correct += (predicted.view(-1) == targets).sum().item()

      # Calculate the average loss over the entire training data
      avg_train_loss = total_loss / len(trainloader)
      avg_train_acc = correct/total

      print('Epoch %d Train Loss: %.3f | Train Accuracy: %.3f' % (epoch + 1, avg_train_loss, avg_train_acc))

    # Process is complete.
    print('Training process has finished. Saving trained model.')
    print('Starting testing')

    # Saving the model
    save_path = f'/content/modelsru-fold-{fold+1}.pth'
    torch.save(network.state_dict(), save_path)
    # torch.save(network, save_path)

    # Evaluation for this fold
    correct, total = 0, 0
    predicted_labels = []
    true_labels = []
    network.eval()
    with torch.no_grad():
      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = network(inputs)

        predicted = (outputs > 0.5).float()
        total += targets.size(0)
        correct += (predicted.view(-1) == targets).sum().item()

        predicted_labels.extend(predicted.tolist())
        true_labels.extend(targets.tolist())

      # Print metrics
      accuracy = accuracy_score(true_labels, predicted_labels)
      precision = precision_score(true_labels, predicted_labels)
      recall = recall_score(true_labels, predicted_labels)
      f1 = f1_score(true_labels, predicted_labels)
      cm = confusion_matrix(true_labels, predicted_labels)

      print('Accuracy for fold %d: %.4f' % (fold+1, accuracy))
      print('Precision for fold %d: %.4f' % (fold+1, precision))
      print('Recall for fold %d: %.4f' % (fold+1, recall))
      print('F1-Score for fold %d: %.4f' % (fold+1, f1))
      print('Confusion Matrix for fold', fold+1, '\n', cm)
      print("-"*60)

      results[fold] = {
          'accuracy': accuracy,
          'precision': precision,
          'recall': recall,
          'f1': f1,
          'cm': cm
      }

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print("-"*60)

  sum_accuracy = []
  sum_precision = []
  sum_recall = []
  sum_f1 = []

  for fold, metrics in results.items():
      accuracy = metrics['accuracy']
      precision = metrics['precision']
      recall = metrics['recall']
      f1 = metrics['f1']
      cm = metrics['cm']

      print(f'Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')
      print(cm)

      sum_accuracy.append(accuracy)
      sum_precision.append(precision)
      sum_recall.append(recall)
      sum_f1.append(f1)

  average_accuracy = np.mean(sum_accuracy)
  std_accuracy = np.std(sum_accuracy)

  average_precision = np.mean(sum_precision)
  std_precision = np.std(sum_precision)

  average_recall = np.mean(sum_recall)
  std_recall = np.std(sum_recall)

  average_f1 = np.mean(sum_f1)
  std_f1 = np.std(sum_f1)

  print("-"*60)
  print(f'Average - Accuracy: {average_accuracy*100:.2f}% (+/- {std_accuracy*100:.2f}%), Precision: {average_precision*100:.2f}% (+/- {std_precision*100:.2f}%), Recall: {average_recall*100:.2f}% (+/- {std_recall*100:.2f}%), F1-Score: {average_f1*100:.2f}% (+/- {std_f1*100:.2f}%)')

# CV

In [ ]:
for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset_tensor.tensors[0], dataset_tensor.tensors[1])):
    # Print
    print(f'FOLD {fold+1}')
    print('--------------------------------')
    # Mengakses label/train target di setiap lipatan
    train_labels = dataset_tensor.tensors[1][train_ids]
    val_labels = dataset_tensor.tensors[1][val_ids]

    # Menghitung jumlah label di setiap lipatan
    num_train_labels = len(train_labels)
    num_val_labels = len(val_labels)

    # Menghitung jumlah label positif dan negatif di setiap lipatan pelatihan
    num_pos_train = (train_labels == 1).sum()
    num_neg_train = (train_labels == 0).sum()

    # Menghitung jumlah label positif dan negatif di setiap lipatan validasi
    num_pos_val = (val_labels == 1).sum()
    num_neg_val = (val_labels == 0).sum()

    print(f'Jumlah label negatif di set pelatihan: {num_neg_train}')
    print(f'Jumlah label positif di set pelatihan: {num_pos_train}')
    print(f'Jumlah label negatif di set validasi: {num_neg_val}')
    print(f'Jumlah label positif di set validasi: {num_pos_val}')

FOLD 1
--------------------------------
Jumlah label negatif di set pelatihan: 1061
Jumlah label positif di set pelatihan: 857
Jumlah label negatif di set validasi: 265
Jumlah label positif di set validasi: 215
FOLD 2
--------------------------------
Jumlah label negatif di set pelatihan: 1061
Jumlah label positif di set pelatihan: 857
Jumlah label negatif di set validasi: 265
Jumlah label positif di set validasi: 215
FOLD 3
--------------------------------
Jumlah label negatif di set pelatihan: 1060
Jumlah label positif di set pelatihan: 858
Jumlah label negatif di set validasi: 266
Jumlah label positif di set validasi: 214
FOLD 4
--------------------------------
Jumlah label negatif di set pelatihan: 1061
Jumlah label positif di set pelatihan: 858
Jumlah label negatif di set validasi: 265
Jumlah label positif di set validasi: 214
FOLD 5
--------------------------------
Jumlah label negatif di set pelatihan: 1061
Jumlah label positif di set pelatihan: 858
Jumlah label negatif di set v

In [ ]:
for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset_tensor.tensors[0], dataset_tensor.tensors[1])):
    print(f'Fold:{fold+1}, Train set: {len(train_ids)}, Test set:{len(val_ids)}')
    fold+=1

Fold:1, Train set: 1918, Test set:480
Fold:2, Train set: 1918, Test set:480
Fold:3, Train set: 1918, Test set:480
Fold:4, Train set: 1919, Test set:479
Fold:5, Train set: 1919, Test set:479


# Training Model

## **LSTM**

In [31]:
set_seed(42)
lstm_model, optimizer_lstm = initilize_lstm_model(pretrained_embedding=embeddings,
                                               hidden_dim=128,
                                               dropout=0.3,
                                               learning_rate=0.001,
                                              bidirectional=False)

train_cv(lstm_model, optimizer_lstm, dataset_tensor, k_folds, batch_size=32, num_epochs=30)

------------------------------------------------------------
FOLD 1
------------------------------------------------------------
Epoch 1 Train Loss: 0.661 | Train Accuracy: 0.619
Epoch 2 Train Loss: 0.599 | Train Accuracy: 0.681
Epoch 3 Train Loss: 0.572 | Train Accuracy: 0.714
Epoch 4 Train Loss: 0.552 | Train Accuracy: 0.723
Epoch 5 Train Loss: 0.529 | Train Accuracy: 0.738
Epoch 6 Train Loss: 0.523 | Train Accuracy: 0.739
Epoch 7 Train Loss: 0.514 | Train Accuracy: 0.761
Epoch 8 Train Loss: 0.505 | Train Accuracy: 0.748
Epoch 9 Train Loss: 0.494 | Train Accuracy: 0.765
Epoch 10 Train Loss: 0.485 | Train Accuracy: 0.779
Epoch 11 Train Loss: 0.488 | Train Accuracy: 0.770
Epoch 12 Train Loss: 0.475 | Train Accuracy: 0.779
Epoch 13 Train Loss: 0.463 | Train Accuracy: 0.787
Epoch 14 Train Loss: 0.463 | Train Accuracy: 0.779
Epoch 15 Train Loss: 0.445 | Train Accuracy: 0.797
Epoch 16 Train Loss: 0.441 | Train Accuracy: 0.794
Epoch 17 Train Loss: 0.442 | Train Accuracy: 0.797
Epoch 18 Trai

In [ ]:
lstm_model

LSTMModel(
  (embedding): Embedding(3650, 300)
  (dropout): Dropout(p=0.3, inplace=False)
  (lstm): LSTM(300, 150, batch_first=True)
  (avg_pooling): AdaptiveAvgPool1d(output_size=1)
  (max_pooling): AdaptiveMaxPool1d(output_size=1)
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## **BiLSTM**

In [32]:
set_seed(42)
bilstm_model, optimizer_bilstm = initilize_lstm_model(pretrained_embedding=embeddings,
                                               hidden_dim=128,
                                               dropout=0.3,
                                               learning_rate=0.001,
                                              bidirectional=True)
train_cv(bilstm_model, optimizer_bilstm, dataset_tensor, k_folds, batch_size=32, num_epochs=30)

------------------------------------------------------------
FOLD 1
------------------------------------------------------------
Epoch 1 Train Loss: 0.652 | Train Accuracy: 0.633
Epoch 2 Train Loss: 0.604 | Train Accuracy: 0.676
Epoch 3 Train Loss: 0.578 | Train Accuracy: 0.709
Epoch 4 Train Loss: 0.558 | Train Accuracy: 0.719
Epoch 5 Train Loss: 0.535 | Train Accuracy: 0.733
Epoch 6 Train Loss: 0.517 | Train Accuracy: 0.750
Epoch 7 Train Loss: 0.518 | Train Accuracy: 0.741
Epoch 8 Train Loss: 0.507 | Train Accuracy: 0.747
Epoch 9 Train Loss: 0.498 | Train Accuracy: 0.759
Epoch 10 Train Loss: 0.481 | Train Accuracy: 0.776
Epoch 11 Train Loss: 0.480 | Train Accuracy: 0.772
Epoch 12 Train Loss: 0.471 | Train Accuracy: 0.771
Epoch 13 Train Loss: 0.463 | Train Accuracy: 0.785
Epoch 14 Train Loss: 0.455 | Train Accuracy: 0.791
Epoch 15 Train Loss: 0.447 | Train Accuracy: 0.788
Epoch 16 Train Loss: 0.449 | Train Accuracy: 0.788
Epoch 17 Train Loss: 0.439 | Train Accuracy: 0.795
Epoch 18 Trai

In [ ]:
bilstm_model

LSTMModel(
  (embedding): Embedding(3650, 300)
  (dropout): Dropout(p=0.3, inplace=False)
  (lstm): LSTM(300, 150, batch_first=True, bidirectional=True)
  (avg_pooling): AdaptiveAvgPool1d(output_size=1)
  (max_pooling): AdaptiveMaxPool1d(output_size=1)
  (fc): Linear(in_features=600, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## **GRU**

In [33]:
set_seed(42)
gru_model, optimizer_gru = initilize_gru_model(pretrained_embedding=embeddings,
                                               hidden_dim=128,
                                               dropout=0.3,
                                               learning_rate=0.001,
                                              bidirectional=False)

train_cv(gru_model, optimizer_gru, dataset_tensor, k_folds, batch_size=32, num_epochs=30)

------------------------------------------------------------
FOLD 1
------------------------------------------------------------
Epoch 1 Train Loss: 0.684 | Train Accuracy: 0.605
Epoch 2 Train Loss: 0.603 | Train Accuracy: 0.688
Epoch 3 Train Loss: 0.560 | Train Accuracy: 0.722
Epoch 4 Train Loss: 0.543 | Train Accuracy: 0.732
Epoch 5 Train Loss: 0.527 | Train Accuracy: 0.740
Epoch 6 Train Loss: 0.501 | Train Accuracy: 0.747
Epoch 7 Train Loss: 0.501 | Train Accuracy: 0.768
Epoch 8 Train Loss: 0.481 | Train Accuracy: 0.761
Epoch 9 Train Loss: 0.475 | Train Accuracy: 0.778
Epoch 10 Train Loss: 0.467 | Train Accuracy: 0.786
Epoch 11 Train Loss: 0.449 | Train Accuracy: 0.786
Epoch 12 Train Loss: 0.445 | Train Accuracy: 0.794
Epoch 13 Train Loss: 0.425 | Train Accuracy: 0.813
Epoch 14 Train Loss: 0.423 | Train Accuracy: 0.811
Epoch 15 Train Loss: 0.411 | Train Accuracy: 0.825
Epoch 16 Train Loss: 0.403 | Train Accuracy: 0.823
Epoch 17 Train Loss: 0.397 | Train Accuracy: 0.831
Epoch 18 Trai

In [ ]:
gru_model

GRUModel(
  (embedding): Embedding(3650, 300)
  (dropout): Dropout(p=0.3, inplace=False)
  (gru): GRU(300, 150, batch_first=True)
  (avg_pooling): AdaptiveAvgPool1d(output_size=1)
  (max_pooling): AdaptiveMaxPool1d(output_size=1)
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## **BiGRU**

In [34]:
set_seed(42)
bigru_model, optimizer_bigru = initilize_gru_model(pretrained_embedding=embeddings,
                                               hidden_dim=128,
                                               dropout=0.3,
                                               learning_rate=0.001,
                                              bidirectional=True)

train_cv(bigru_model, optimizer_bigru, dataset_tensor, k_folds, batch_size=32, num_epochs=30)

------------------------------------------------------------
FOLD 1
------------------------------------------------------------
Epoch 1 Train Loss: 0.708 | Train Accuracy: 0.604
Epoch 2 Train Loss: 0.606 | Train Accuracy: 0.686
Epoch 3 Train Loss: 0.577 | Train Accuracy: 0.706
Epoch 4 Train Loss: 0.548 | Train Accuracy: 0.726
Epoch 5 Train Loss: 0.532 | Train Accuracy: 0.734
Epoch 6 Train Loss: 0.528 | Train Accuracy: 0.735
Epoch 7 Train Loss: 0.501 | Train Accuracy: 0.765
Epoch 8 Train Loss: 0.496 | Train Accuracy: 0.763
Epoch 9 Train Loss: 0.481 | Train Accuracy: 0.772
Epoch 10 Train Loss: 0.470 | Train Accuracy: 0.782
Epoch 11 Train Loss: 0.463 | Train Accuracy: 0.778
Epoch 12 Train Loss: 0.450 | Train Accuracy: 0.791
Epoch 13 Train Loss: 0.446 | Train Accuracy: 0.798
Epoch 14 Train Loss: 0.432 | Train Accuracy: 0.802
Epoch 15 Train Loss: 0.429 | Train Accuracy: 0.816
Epoch 16 Train Loss: 0.424 | Train Accuracy: 0.811
Epoch 17 Train Loss: 0.414 | Train Accuracy: 0.818
Epoch 18 Trai

In [ ]:
bigru_model

GRUModel(
  (embedding): Embedding(3650, 300)
  (dropout): Dropout(p=0.3, inplace=False)
  (gru): GRU(300, 150, num_layers=2, batch_first=True, bidirectional=True)
  (avg_pooling): AdaptiveAvgPool1d(output_size=1)
  (max_pooling): AdaptiveMaxPool1d(output_size=1)
  (fc): Linear(in_features=600, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## **SRU**

In [43]:
set_seed(42)
sru_model, optimizer_sru = initilize_sru_model(pretrained_embedding=embeddings,
                                               hidden_dim=128,
                                               dropout=0.3,
                                               learning_rate=0.001,
                                               bidirectional=False)

train_cv(sru_model, optimizer_sru, dataset_tensor, k_folds, batch_size=32, num_epochs=30)

------------------------------------------------------------
FOLD 1
------------------------------------------------------------
Epoch 1 Train Loss: 0.829 | Train Accuracy: 0.609
Epoch 2 Train Loss: 0.598 | Train Accuracy: 0.687
Epoch 3 Train Loss: 0.572 | Train Accuracy: 0.704
Epoch 4 Train Loss: 0.546 | Train Accuracy: 0.722
Epoch 5 Train Loss: 0.521 | Train Accuracy: 0.749
Epoch 6 Train Loss: 0.505 | Train Accuracy: 0.759
Epoch 7 Train Loss: 0.485 | Train Accuracy: 0.762
Epoch 8 Train Loss: 0.478 | Train Accuracy: 0.773
Epoch 9 Train Loss: 0.474 | Train Accuracy: 0.777
Epoch 10 Train Loss: 0.462 | Train Accuracy: 0.778
Epoch 11 Train Loss: 0.447 | Train Accuracy: 0.796
Epoch 12 Train Loss: 0.435 | Train Accuracy: 0.799
Epoch 13 Train Loss: 0.429 | Train Accuracy: 0.800
Epoch 14 Train Loss: 0.428 | Train Accuracy: 0.797
Epoch 15 Train Loss: 0.418 | Train Accuracy: 0.810
Epoch 16 Train Loss: 0.408 | Train Accuracy: 0.810
Epoch 17 Train Loss: 0.403 | Train Accuracy: 0.817
Epoch 18 Trai

In [40]:
sru_model

SRUModel(
  (embedding): Embedding(3642, 300)
  (dropout): Dropout(p=0.3, inplace=False)
  (sru): SRU(
    (rnn_lst): ModuleList(
      (0): SRUCell(300, 128, rescale=False)
    )
  )
  (avg_pooling): AdaptiveAvgPool1d(output_size=1)
  (max_pooling): AdaptiveMaxPool1d(output_size=1)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## **BiSRU**

In [36]:
set_seed(42)
bisru_model, optimizer_bisru = initilize_sru_model(pretrained_embedding=embeddings,
                                               hidden_dim=128,
                                               dropout=0.3,
                                               learning_rate=0.001,
                                               bidirectional=True)

train_cv(bisru_model, optimizer_bisru, dataset_tensor, k_folds, batch_size=32, num_epochs=30)

------------------------------------------------------------
FOLD 1
------------------------------------------------------------
Epoch 1 Train Loss: 0.906 | Train Accuracy: 0.627
Epoch 2 Train Loss: 0.578 | Train Accuracy: 0.693
Epoch 3 Train Loss: 0.544 | Train Accuracy: 0.713
Epoch 4 Train Loss: 0.527 | Train Accuracy: 0.748
Epoch 5 Train Loss: 0.507 | Train Accuracy: 0.753
Epoch 6 Train Loss: 0.494 | Train Accuracy: 0.771
Epoch 7 Train Loss: 0.475 | Train Accuracy: 0.777
Epoch 8 Train Loss: 0.472 | Train Accuracy: 0.786
Epoch 9 Train Loss: 0.455 | Train Accuracy: 0.792
Epoch 10 Train Loss: 0.443 | Train Accuracy: 0.795
Epoch 11 Train Loss: 0.443 | Train Accuracy: 0.787
Epoch 12 Train Loss: 0.429 | Train Accuracy: 0.802
Epoch 13 Train Loss: 0.419 | Train Accuracy: 0.806
Epoch 14 Train Loss: 0.408 | Train Accuracy: 0.816
Epoch 15 Train Loss: 0.401 | Train Accuracy: 0.817
Epoch 16 Train Loss: 0.410 | Train Accuracy: 0.804
Epoch 17 Train Loss: 0.386 | Train Accuracy: 0.833
Epoch 18 Trai